In [ ]:
import tweepy 
from tweepy import Stream 
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

In [ ]:
CONSUMER_KEY = "consumer_key"
CONSUMER_SECRET = "consumer_secret"
ACCESS_TOKEN = "access_token"
ACCESS_SECRET = "access_secret"

In [ ]:
keyword = ['covid-19', 'corona', 'covid', 'coronavirus', 'covid-19 india', 'coronavirus india']

In [ ]:
class TwitterStreamer(StreamListener):
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:
            jdata = json.loads(data)
            print("New data")
            if "extended_tweet" in jdata:
                self.client_socket.send(data.encode())
                #print(jdata['extended_tweet']['full_text'])
            else:
                self.client_socket.send(data.encode())
                #print(jdata['text'])
            print(jdata['coordinates'])
            return True
        except BaseException as e:
            print('Error on_data %s' %str(e))
        return True
    
    def on_error(self, status):
        print(status)
        return True
              

In [ ]:
def sendData(c_socket, keyword):
    print('Start sending data from Twitter to Socket')
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    stream = Stream(auth, TwitterStreamer(c_socket))
    stream.filter(track=keyword, languages=['en'])

In [ ]:
s = socket.socket()
host = "localhost"
port = 5555
s.bind((host, port))
print('Socket is ready')
s.listen(4)
print('Socket is listening')
csocket, addr = s.accept()
print('Received request from: ' + str(addr))
sendData(csocket, keyword)